##![LearnAI Header](https://coursematerial.blob.core.windows.net/assets/LearnAI_header.png)

# Hyperparameter tuning using Random Forests

In this lab, we are going to use a [Random Forest](https://en.wikipedia.org/wiki/Random_forest) in combination with a parameter grid and cross-validation to perform hyperparameter tuning to improve our solution for Predictive Maintenance.

Additional Documentation:
- Spark API documentation for [Random Forests](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestClassifier)
- Spark ML Guide for [Random Forests](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier)

## Read the data

Let's load and cache the data once more.

In [5]:
df = spark.read.parquet("dbfs:/FileStore/tables/preprocessed").cache()
df = df.withColumnRenamed("y_0", "label") # this is the label we're going to use

display(df)

machineID,datetime,age,diff_error_0,diff_error_1,diff_error_2,diff_error_3,diff_error_4,diff_fail_0,diff_fail_1,diff_fail_2,diff_fail_3,diff_maint_0,diff_maint_1,diff_maint_2,diff_maint_3,pressure_ma_3,pressure_sd_3,rotate_ma_3,rotate_sd_3,vibration_ma_3,vibration_sd_3,volt_ma_3,volt_sd_3,label,y_1,y_2,y_3
32,2015-02-23T21:00:00.000+0000,15,1095.0,166.0,409.0,1387.0,375.0,1071.0,1387.0,1387.0,351.0,711.0,1387.0,1387.0,351.0,99.47241720086753,4.027627389831299,451.38980736669623,37.143235255898006,36.437978246330076,5.246946040863718,175.77508414243,27.49122867502091,0,0,0,0
32,2015-02-23T22:00:00.000+0000,15,1096.0,167.0,410.0,1388.0,376.0,1072.0,1388.0,1388.0,352.0,712.0,1388.0,1388.0,352.0,100.09894109745454,3.794141420379903,435.17321536069,39.335699599244414,39.2308973568302,4.176148610708063,180.83567404661,23.740298384283044,0,0,0,0
32,2015-02-23T23:00:00.000+0000,15,1097.0,168.0,411.0,1389.0,377.0,1073.0,1389.0,1389.0,353.0,713.0,1389.0,1389.0,353.0,99.54439890229081,3.4758830167295156,421.66272855994526,15.279105023168727,41.711212313841905,0.9869915716793245,169.48155636960098,21.844321498131357,0,0,0,0
32,2015-02-24T00:00:00.000+0000,15,1098.0,169.0,412.0,1390.0,378.0,1074.0,1390.0,1390.0,354.0,714.0,1390.0,1390.0,354.0,97.68908923167595,2.826179881000501,416.56100286652577,14.313070539126413,43.106649846394575,2.4962971618940304,168.40722316791698,19.978049808200037,0,0,0,0
32,2015-02-24T01:00:00.000+0000,15,1099.0,170.0,413.0,1391.0,379.0,1075.0,1391.0,1391.0,355.0,715.0,1391.0,1391.0,355.0,96.91916094721387,4.036157929664655,428.15124045533855,23.48933395357947,42.8987242528714,2.771261929609087,172.9402295425665,16.124035462907294,0,0,0,0
32,2015-02-24T02:00:00.000+0000,15,1100.0,171.0,414.0,1392.0,380.0,1076.0,1392.0,1392.0,356.0,716.0,1392.0,1392.0,356.0,98.60820305322397,6.429119080028224,444.72279625965575,26.725216590298036,43.55483208223855,2.8184999168524905,166.95757356868276,19.57722439213211,0,0,0,0
32,2015-02-24T03:00:00.000+0000,15,1101.0,172.0,415.0,1393.0,381.0,1077.0,1393.0,1393.0,357.0,717.0,1393.0,1393.0,357.0,97.75829917013185,6.35926449953383,455.000717946228,30.802700661866158,46.0032332920517,5.284176271790364,171.5146198226785,18.02024867702125,0,0,0,0
32,2015-02-24T04:00:00.000+0000,15,1102.0,173.0,416.0,1394.0,382.0,1078.0,1394.0,1394.0,358.0,718.0,1394.0,1394.0,358.0,97.46746997114926,6.507590246654807,464.63403314151725,13.781545445572954,44.00402649786548,6.376598773584249,163.08366794565802,10.130385844311215,0,0,0,0
32,2015-02-24T05:00:00.000+0000,15,1103.0,174.0,417.0,1395.0,383.0,1079.0,1395.0,1395.0,359.0,719.0,1395.0,1395.0,359.0,103.47412855441392,9.726682380322783,481.54989358335945,31.29419249089553,41.575592514816776,9.504942586795327,164.627163319354,11.547022705888775,0,0,0,0
32,2015-02-24T06:00:00.000+0000,15,1104.0,175.0,418.0,1396.0,384.0,1080.0,1396.0,1396.0,360.0,720.0,0.0,1396.0,360.0,99.97419460128005,10.652800910349333,479.901478683917,32.041360774820994,39.010816091632826,9.76504784353832,168.14560647024325,6.929237687861469,0,0,0,0


In [6]:
keys = ['machineID', 'datetime']
X_keep = ['diff_maint_1', 'diff_error_1', 'volt_sd_3', 'diff_fail_3', 'pressure_ma_3', 'pressure_sd_3', 'diff_fail_1', 'diff_fail_0', 'age', 'vibration_ma_3', 'rotate_ma_3', 'diff_error_2', 'diff_fail_2', 'diff_error_3', 'diff_maint_2', 'volt_ma_3', 'diff_maint_0', 'vibration_sd_3', 'diff_maint_3', 'rotate_sd_3', 'diff_error_0', 'diff_error_4']
Y_keep = ['y_0', 'y_1', 'y_2', 'y_3']

In [7]:
from pandas import datetime
from pyspark.sql.functions import col, hour

# we sample every nth row of the data using the `hour` function
df_train = df.filter((col('datetime') < datetime(2015, 10, 1))) # & (hour(col('datetime')) % 3 == 0))
df_test = df.filter(col('datetime') > datetime(2015, 10, 15))

In [8]:
display(df_train.describe())

summary,machineID,age,diff_error_0,diff_error_1,diff_error_2,diff_error_3,diff_error_4,diff_fail_0,diff_fail_1,diff_fail_2,diff_fail_3,diff_maint_0,diff_maint_1,diff_maint_2,diff_maint_3,pressure_ma_3,pressure_sd_3,rotate_ma_3,rotate_sd_3,vibration_ma_3,vibration_sd_3,volt_ma_3,volt_sd_3,label,y_1,y_2,y_3
count,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600,654600
mean,50.5,11.33,767.0613458600673,998.2097525206233,933.592835319279,1020.7992407577146,1615.1251894286588,2133.154399633364,1935.9317461044914,2652.4212694775433,2349.7094821264895,1113.8832447296058,1109.771384051329,1144.6514573785519,1120.9993217231897,100.82513452018438,9.23999324497866,446.6710800808024,46.207254381075714,40.396483722277594,4.619880429306799,170.79153282266068,13.81533582506386,0.014637946837763519,0.01884967919340055,0.010820348304307975,0.015073327222731438
stddev,28.866092096380157,5.8276197440095325,763.1257842489293,955.9490918887524,994.1383169517087,995.1194154164116,1427.1820509876297,1655.1293083917917,1552.799216609115,1935.571758820081,1866.5733315385494,1000.6749291181277,1023.5238622627411,1038.5993590886062,1038.328199929404,6.78311055919519,3.916499473522474,29.659338215200368,19.486555315830316,3.1722981813853623,1.9533044798825747,8.448016911894165,5.845427117512882,0.12009870683945567,0.13599410663951897,0.10345668039213028,0.1218447565916889
min,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0053324677532,0.09369705911616336,198.770337377762,0.21592334380224446,22.9732894818614,0.016413819956125182,135.9794565589355,0.0299919681797135,0,0,0,0
max,100,20,5482.0,6645.0,6645.0,6617.0,6645.0,6645.0,6645.0,6645.0,6645.0,6645.0,6645.0,6645.0,6645.0,164.87532404477776,32.69926439338247,576.923563024815,164.91776461098644,67.31208520010537,15.995336982084046,233.13577630387576,54.584229250416726,1,1,1,1


## Define your Random Forest

Random forests and ensembles of decision trees are more powerful than a single decision tree alone. 

Let's take a look at all the hyperparameters we could change in [RandomForestClassifier](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.RandomForestClassifier).

Or run the next cell to get an explanation of the hyperparameters for the Random Forest algorithm.

In [10]:
from pyspark.ml.classification import RandomForestClassifier

RandomForestClassifier().explainParams()

Out[ 5 ]: "cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)\nfeatureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: auto)\nfeaturesCol: features column name. (default: features)\nimpurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)\nlabelCol: label column name. (default: label)\nmaxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)\nmaxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)\nmaxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)\nminInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)\nminInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)\nnumTrees: Number of trees to train (>= 1). (default: 20)\npredictionCol: prediction column name. (default: prediction)\nprobabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)\nseed: random seed. (default: -184448601186841989)\nsubsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)"

## Let's define a Pipeline

Now that we have all of the feature transformations and estimators set up, let's put all of the stages together in the pipline.

In [12]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

vassembler = VectorAssembler(inputCols = X_keep, outputCol = "features")

rf = (RandomForestClassifier()
      .setLabelCol("label")
      .setFeaturesCol("features")
      .setSeed(27))

pipeline = Pipeline(stages = [vassembler, rf])

### Hands-on lab
Try changing the values of `numTrees` and `maxDepth` to any values you like

HINT: Take a look at the docs

In [14]:
#todo

In [15]:
# maximize this cell to see the solution:
  
rf.setNumTrees(100)
rf.setMaxDepth(10)

Out[ 8 ]: RandomForestClassifier_befa1fa0433e

### End of lab

In [17]:
pipeline.getStages()

Out[ 9 ]: [VectorAssembler_f1630a46be5c, RandomForestClassifier_befa1fa0433e]

If you want to look at what parameter each stage in the pipeline takes.

In [19]:
pipeline.getStages()[0].extractParamMap()

Out[ 10 ]: 
{Param(parent='VectorAssembler_f1630a46be5c', name='handleInvalid', doc="How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip')."): 'error',
 Param(parent='VectorAssembler_f1630a46be5c', name='inputCols', doc='input column names.'): ['diff_maint_1',
 'diff_error_1',
 'volt_sd_3',
 'diff_fail_3',
 'pressure_ma_3',
 'pressure_sd_3',
 'diff_fail_1',
 'diff_fail_0',
 'age',
 'vibration_ma_3',
 'rotate_ma_3',
 'diff_error_2',
 'diff_fail_2',
 'diff_error_3',
 'diff_maint_2',
 'volt_ma_3',
 'diff_maint_0',
 'vibration_sd_3',
 'diff_maint_3',
 'rotate_sd_3',
 'diff_error_0',
 'diff_error_4'],
 Param(parent='VectorAssembler_f1630a46be5c', name='outputCol', doc='output column name.'): 'features'}

## Setup the grid for searching hyperparameters

There are a lot of hyperparamaters we could tune, and it would take a long time to manually configure.

Instead of a manual (ad-hoc) approach, let's use Spark's [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) to find the optimal hyperparameters in a more systematic approach.

In this example notebook, we keep these trees shallow and use a relatively small number of trees. Let's define a grid of hyperparameters to test:
  - maxDepth: max depth of each decision tree in the RF ensemble (Use the values `2, 5, 10`)
  - numTrees: number of trees in each RF ensemble (Use the values `10, 50`)

In [21]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
#             .addGrid()
#             .addGrid()
            .build())

In [22]:
# maximize this cell to see the solution:

from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [3, 5, 10]) 
            .addGrid(rf.numTrees, [5, 10, 20]) 
            .build())

## Define cross-validation procedure

As in the previous lab for *sentiment analysis*, we are going to use 3-fold cross validation.

We pass in the `estimator` (pipeline), `evaluator`, and `estimatorParamMaps` to [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) so that it knows:
- Which model to use
- How to evaluate the model
- What hyperparamters to set for the model

We can also set the number of folds we want to split our data into (3), as well as setting a seed so we all have the same split in the data.

In [25]:
from pyspark.ml.evaluation import  BinaryClassificationEvaluator

from pyspark.ml.tuning import CrossValidator

evaluator = (BinaryClassificationEvaluator()
             .setLabelCol("label")
             .setRawPredictionCol("rawPrediction")
             .setMetricName("areaUnderROC"))

cv = (CrossValidator()
      .setEstimator(pipeline) # our pipeline for building a random forest model
      .setEvaluator(evaluator) # the evaluator let's CV know how to evaluate models
      .setEstimatorParamMaps(paramGrid) # the hyper-pramaters we're iterating over
      .setNumFolds(3) # every hyper-parameter combination is evaluated using cross-validaton with 3-folds 
      .setSeed(27)) # we set the seed for reproducibilty

In [26]:
cv_rf_model = cv.fit(df_train)

Let's take a look at the model with the best hyperparameter configuration

In [28]:
results = list(zip(cv_rf_model.getEstimatorParamMaps(), cv_rf_model.avgMetrics))

for x in results:
  print(x[0].values(), x[1])

dict_values([3, 5]) 0.6052002822326324
dict_values([5, 5]) 0.9040217073464585
dict_values([10, 5]) 0.9748806472070224
dict_values([3, 10]) 0.8088097454194326
dict_values([5, 10]) 0.9277400376078913
dict_values([10, 10]) 0.9765763939422862
dict_values([3, 20]) 0.8543349570696941
dict_values([5, 20]) 0.930211531735222
dict_values([10, 20]) 0.9815567173806785

## Evaluate the result of your cross-validated hyperparameter tuning

Let's see how well we did on the test set.

In [30]:
df_pred = cv_rf_model.bestModel.transform(df_test)
display(df_pred.groupBy("label", "prediction").count())

label,prediction,count
1,0.0,1518
0,0.0,185641
1,1.0,570
0,1.0,71


In [31]:
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol("label")
evaluator.setRawPredictionCol("rawPrediction")

auroc = evaluator.setMetricName("areaUnderROC").evaluate(df_pred)
print("AUROC: {}".format(auroc))

AUROC: 0.9453818032770718

Once again let's save our pipeline in case we need to make future use of it.

In [33]:
# cv_rf_model.write().overwrite().save("/dbfs/tmp/cv_rf_model_ipeline")

Interesting. The results are quite a bit lower for the test set than they were during cross-validation. Can you say why?

In [35]:
# You can ignore this code, we use it for testing our notebooks.
assert auroc > .9

#### Improving our model

You are not done yet!  There are several ways we could further improve our model:
* **Expert knowledge** 
* **Better tuning** 
* **Feature engineering**

As an exercise: Replace the Random Forest code with a Gradient Boosted tree, and vary the number of trees and depth of the trees. What do you find?

*Good luck!*